## Imports

In [1]:
from pathlib import Path # reads paths in the current OS
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import plot_utils as pu
import preprocess as prep

In [2]:
execute = prep.check_output_files()
if execute == 'y':
    !python get_data_ready.py

It looks like you have all the necessary files for the analysis.
Do you want to execute the process anyway and update them? (y = yes, n = no): y
Reading the initial files
Getting "comarca" column ready for each file
	Comarca not found: Repartim al Bages, Solsonès, Barcelonès i Berguedà
Pre-processing columns
Add new columns to the datasets
Find the duplicates in the datasets and merging the data
	 Partial match: BdeGust Beer Cervesa Compromesa vs. BdeGust Cervesa Compromesa . Match ratio: 91 , partial ratio: 81
	 Partial match: BIOGRASSFEED vs. Biograssfed . Match ratio: 96 , partial ratio: 91
	 Partial match: Tomating vs. Tomàting . Match ratio: 88 , partial ratio: 88
	 Partial match: Ramaderia Ecològica ca l'Andreu vs. Ramaderia ecológica ca l’andreu . Match ratio: 94 , partial ratio: 94
	 Partial match: L'Enllaç de l'Ebre vs. L'Enllaç de l’Ebre, SCCL . Match ratio: 79 , partial ratio: 89
	 Partial match: (Mu)DA Vedella d'Agullana vs. MuDA. Vedella d'Agullana . Match ratio: 94 , par

	 Partial match: COOPERATIVA AGRÍCOLA I CA SANT ISIDRE vs. Cooperativa Agricola Sant Isidre Del Perelló . Match ratio: 77 , partial ratio: 84
	 Partial match: EL TROS D'ORDAL, S.L. vs. El Tros dOrdal SL . Match ratio: 87 , partial ratio: 89
	 Partial match: ESPELT VITICULTORS, SL vs. Mulet Viticultors . Match ratio: 72 , partial ratio: 82
	 Partial match: FAMILIA ESTEVE RAFOLS, SCP vs. Família Esteve Ràfols SCP . Match ratio: 90 , partial ratio: 88
	 Partial match: FORMATGES ARTESANS DE PONENT SCP vs. FORMATGES ARTESANS DE PONENT,S L . Match ratio: 91 , partial ratio: 91
	 Partial match: FORMATGES ARTESANS DE PONENT, SL vs. FORMATGES ARTESANS DE PONENT,S L . Match ratio: 97 , partial ratio: 97
	 Partial match: GRANJA EL ROCALL, S.L. vs. Granja la Roca . Match ratio: 72 , partial ratio: 93
	 Partial match: HORT DE LA CISTELLERA, SCP vs. HORT DE CA LA CISTELLERA . Match ratio: 84 , partial ratio: 88
	 Partial match: LA CABANA D'ALCOVER,SL vs. La Cabana d'Alcover SL . Match ratio: 95 , p

## Reading

In [3]:
data, vdp, com_coord, cat = prep.read_final_data()

In [ ]:
data.shape

In [ ]:
aux_text = {'total':'en total',
            'n_abastiment':'abastiment',
            'n_pagesos':'pagesos',
            'vegetables':'de verdures',
           'fruit':'de fruita',
           'meat':'de carn',
           'iseco':'ecos',
           'n_comarcas_delivery':'comarques amb enviament'}
n_columns = ['n_main_prod','n_other_prod','n_tot_prod','n_paym_methods','n_comarcas_delivery']
n_columns_red = ['n_main_prod','n_other_prod','n_tot_prod']
aux_text_red = {'total':'en total',
            'vegetables':'de verdures',
           'fruit':'de fruita',
           'meat':'de carn'}
max_size = 40
x_low,x_up = 0.18,3.3
y_low,y_up = 40.5,42.9

## Abastiment + pagesos producers in Venta de Proximitat dataset

In [ ]:
ds_intersection = data[data.marca_vdp != '']
ds_intersection.shape

## Comparing abastiment+pagesos and venta de proximitat

In [ ]:
to_plot_vdp = pu.dataset_to_plot(vdp,vdp,com_coord,n_columns_red,multiple_origins=False)
to_plot = pu.dataset_to_plot(data,vdp,com_coord,n_columns,multiple_origins=True)

for col,txt in aux_text_red.items():
    #max_value = max(to_plot_vdp[col].max(),to_plot[col].max())
    fig = pu.plot_map_comarca_points(to_plot,cat,col,txt,max_size,to_plot[col].max(),
                                     x_low,x_up,y_low,y_up,txt.replace('en ','').replace('de ', '').title())
    pu.add_trace_plot(fig,to_plot_vdp,col,txt,max_size,to_plot_vdp[col].max(),
                      '#022d63',series_name='Venda de proximitat')
    #name = config['input_path'] + "/out/" + "comparacio_vdp+covid_map_"+col+".html"
    #fig.write_html(name)
    
    fig.show()

## Noves altes: Abastiment + Pagesos data compared to VdP

In [ ]:
#not in Venta de Proximitat
data_new    = data[data.marca_vdp == '']
to_plot = pu.dataset_to_plot(data_new,vdp,com_coord,n_columns,multiple_origins=True)

In [ ]:
title_name = 'Noves altes per comarca a venta de proximitat degut a la crisi del Covid-19'
fig = pu.create_figure()
fig = pu.add_trace_text_plot(fig,to_plot,'pctge_new','nous',max_size,to_plot['pctge_new'].max(),'#63022d')
fig = pu.plot_layout(fig,cat,x_low,x_up,y_low,y_up,title_name)
fig.write_image("noves_altes_comarca_mapa.png")
fig.write_image("noves_altes_comarca_mapa.pdf")
fig.show()

In [ ]:
title_name = 'Noves altes per comarca a venta de proximitat degut a la crisi del Covid-19'
fig = pu.create_figure()
fig = pu.add_trace_text_plot(fig,to_plot,'total','nous',max_size,to_plot['total'].max(),'#63022d')
fig = pu.plot_layout(fig,cat,x_low,x_up,y_low,y_up,title_name)
fig.write_image("num_noves_altes_comarca_mapa.png")
fig.write_image("num_noves_altes_comarca_mapa.pdf")
fig.show()

## Impacto por sector

In [ ]:
for sector in ('HORTA','FRUITA', 'RAMADERIA','CEREALS', 'ELABORACIO'):

    #only producers of the specific sector
    data_sect=data_new[data_new['is'+sector]==1]
    vdp_sect=vdp[vdp['is'+sector]==1]

    to_plot = ut.dataset_to_plot(data_sect,vdp_sect,com_coord,n_columns,multiple_origins=True)

    title_name = 'SECTOR '+sector+':\n Noves altes per comarca a venta de proximitat degut a la crisi del Covid-19 '
    fig = pu.create_figure()
    fig = pu.add_trace_text_plot(fig,to_plot,'pctge_new','nous',max_size,to_plot['pctge_new'].max(),'#63022d')
    fig = pu.plot_layout(fig,cat,x_low,x_up,y_low,y_up,title_name)
    fig.write_image("noves_altes_comarca_mapa_"+sector+".png")
    fig.show()

    title_name = 'SECTOR '+sector+':\n Noves altes per comarca a venta de proximitat degut a la crisi del Covid-19'
    fig = pu.create_figure()
    fig = pu.add_trace_text_plot(fig,to_plot,'total','nous',max_size,to_plot['total'].max(),'#63022d')
    fig = pu.plot_layout(fig,cat,x_low,x_up,y_low,y_up,title_name)
    fig.write_image("num_noves_altes_comarca_mapa_"+sector+".png")
    fig.show()

## Abastiment + Pagesos data

In [ ]:
to_plot = pu.dataset_to_plot(data,vdp,com_coord,n_columns,multiple_origins=True)

In [ ]:
# plot n not percentages



for col,txt in aux_text.items():
    fig = pu.plot_map_comarca_points(to_plot,cat,col,txt,max_size,to_plot[col].max(),
                                     x_low,x_up,y_low,y_up,txt.replace('en ','').replace('de ', '').title())
    #name = config['input_path'] + "/out/" + "abastiment+pagesos_map_"+col+".html"
    #fig.write_html(name)
    fig.show()

## Abastiment + Pagesos: Logistic connections

In [ ]:
for subset in subset_lists.keys():
    print('-----', subset,'subset -----')
    dic=subset_lists[subset]
        
    on_fields=dic['on_fields']
    off_fields=dic['off_fields']
    label=subset
    
    if on_fields is None:
            if off_fields is None:
                print('There are no subset criteria defined, the whole dataset will be considered')
                data['isSubset']=1
            else:
                data['isSubset']=(np.where(data[off_fields].sum(axis=1).eq(0), 1, 0))
    else:
            print('The sankey diagram will be produced considering only a subset of the whole dataset.')
            print('Especifically, considering just those producers with: ', on_fields, 'on')
            if off_fields is None:
                n_flags_on=len(on_fields)  
                data['isSubset']=(np.where(data[on_fields].sum(axis=1).ge(n_flags_on), 1, 0))
            else:
                print('and the fields: ', off_fields, 'off.')        
                n_flags_off=len(off_fields)
                n_flags_on=len(on_fields)  
                data['flags_on']=(np.where(data[on_fields].sum(axis=1).ge(n_flags_on), 1, 0))
                data['flags_off']=(np.where(data[off_fields].sum(axis=1).eq(0), 1, 0))
                data['isSubset']=np.where(data[['flags_on', 'flags_off']].sum(axis=1).eq(2),1,0)
                data.drop(['flags_on','flags_off'], axis=1, inplace=True) 
                   

    ###-- Selecting only the desired subset of producers
    data_sel=data[(data.isSubset==1) & (~data.comarca_origin.str.contains('NOTFOUND'))]
    if data_sel is None:
        print('There are no producers in the requested subset: ', subset)
        continue
    else:
        print('Dimension of the subset: ', data_sel.shape)
        flows=ut.create_df_for_sankey(data_sel)
        flows=flows[['source', 'target', 'value']] #it's possible to chose another column as weigth,for example norm_value
        ###https://github.com/psychemedia/parlihacks/blob/master/notebooks/MigrantFlow.ipynb)

        SankeyWidget(links=flows.to_dict('records'))
        
        size = dict(width=870, height=1000)

        nodes = {
            'Comarcas_productoras': ProcessGroup(list(locations_df.Comarca.unique())),
            'Comarcas_entrega': ProcessGroup(list(locations_df.Comarca.unique())),
        }

        ordering = [
            ['Comarcas_productoras'],  # put "Comarcas_productoras" on the left...
            ['Comarcas_entrega'],      # ... and "Comarcas_entrega" on the right.
        ]

        bundles = [
            Bundle('Comarcas_productoras', 'Comarcas_entrega'),
        ]

        comarcas_out = Partition.Simple('process',list(locations_df.Comarca.unique()))
        comarcas_in = Partition.Simple('process',list(locations_df.Comarca.unique()))

        # Update the ProcessGroup nodes to use the partitions
        nodes['Comarcas_productoras'].partition = comarcas_out
        nodes['Comarcas_entrega'].partition = comarcas_in

                
        sdd = SankeyDefinition(nodes, bundles, ordering)

        ## New Sankey!
        weave(sdd, flows).to_widget(**size) 

        ## Saving the plot as png
        name = config['input_path'] + "out/" + "SankeyDiag_"+label+".png"

        print('File saved in: ', name)
        display(weave(sdd, flows, link_color=QuantitativeScale('value'), \
          measures='value').to_widget(**size).auto_save_png(name))

## Plots separating dataset origin

### Related to columns with counts

In [ ]:
aux_text = {'n_tot_prod':'Mitja productes totals',
           'n_other_prod':'Mitja productes no principals',
           'n_main_prod':'Mitja productes principals',
            'n_paym_methods':'Mitja métodes de pagament'}
for col,txt in aux_text.items():
    fig = pu.bar_perc_separate_datasets(data, col, txt)
    #name = config['input_path'] + "/out/" + "abastiment+pagesos_bar_"+col+".html"
    #fig.write_html(name)
    fig.show()

In [ ]:
pagament = {
    'targeta':'Targeta',
    'efectiu':'Efectiu',
    'bizum':'Bizum',
    'transferencia previa':'Tranferència',
    'card+cash': 'Efectiu + targeta',
    'trans+cash': 'Efectiu + tranferència',
    'card+trans': 'Transferència + targeta',
    'card+bizum': 'Targeta + Bizum',
    'cash+bizum': 'Efectiu + Bizum',
    'trans+bizum':'Tranferència + Bizum'
}

In [ ]:
data.columns

In [ ]:
ab_gb,pag_gb = pu.pagament_prep(data,pagament)
fig = pu.bar_payment_type(pag_gb,ab_gb)
fig.show()

### Related to other products with some filters

In [ ]:
'''pu.bar_perc_separate_datasets(data[(data.n_main_prod==0) & (data.n_tot_prod>0)],
                         'n_other_prod',aux_text['n_other_prod'])
pu.bar_perc_separate_datasets(data[(data.n_main_prod==1)],'n_other_prod',aux_text['n_other_prod'])
pu.bar_perc_separate_datasets(data[(data.n_main_prod>=2)],'n_other_prod',aux_text['n_other_prod'])'''

Quan no venen productes principals, la majoria venen únicament un producte (no principal).

Quan venen un producte principal, la majoria no venen altre tipus de producte.

Quan venen més d'un producte principal, acostumen a vendre més tipus de productes.